In [9]:
import pandas as pd
from google.cloud import bigquery
from pathlib import Path
import os

cwd = Path(os.getcwd())
PROJECT_ROOT = cwd.parent if cwd.name == "notebooks" else cwd
RAW_DIR = PROJECT_ROOT / "data" / "raw"

def descargar_datos_bigquery_histórico():
    """
    Descarga y concatena datos históricos de ventas desde BigQuery.
    - La primera tabla contiene datos desde 2023 hasta junio de 2025.
    - La segunda tabla contiene datos desde julio de 2025 en adelante.
    El archivo se guarda en data/raw con la fecha del último domingo con datos como nomenclatura.
    """
    from datetime import datetime
    print("Iniciando conexión con BigQuery...")
    client = bigquery.Client()
    print("Conexión establecida.")

    # Descargar datos históricos hasta junio 2025
    query1 = """
    SELECT *
    FROM `fleca-del-port.varios.raw_data_bq_forecasting_20250630`
    """
    df1 = client.query(query1).to_dataframe()

    # Descargar datos desde julio 2025 en adelante
    query2 = """
    SELECT 
        fecha, n_factura, zona_de_venta, producto, familia, cantidad, base_imponible, tipo_IVA, total
    FROM `fleca-del-port.fleca_ventas_dia.t_facturas_dia_extendida_2023`
    WHERE fecha >= '2025-07-01'
    """
    df2 = client.query(query2).to_dataframe()

    # Concatenar ambos DataFrames (unión vertical)
    df = pd.concat([df1, df2], ignore_index=True, sort=False)

    # Calcular el último domingo anterior o igual a la última fecha de datos
    if 'fecha' in df.columns:
        fechas = pd.to_datetime(df['fecha'])
        ultima_fecha = fechas.max()
        ultimo_domingo = ultima_fecha - pd.Timedelta(days=(ultima_fecha.weekday() + 1) % 7)
        fecha_nomenclatura = ultimo_domingo.strftime("%Y%m%d")
        print(f"Rango fechas final: {fechas.min()} - {fechas.max()}")
    else:
        fecha_nomenclatura = datetime.now().strftime("%Y%m%d")

    # Guardar el DataFrame combinado en la carpeta data/raw con nomenclatura de último domingo
    output_path = RAW_DIR / f"raw_data_bq_forecasting_{fecha_nomenclatura}.parquet"
    output_path.parent.mkdir(parents=True, exist_ok=True)
    print(f"Guardando archivo en {output_path} ...")
    df.to_parquet(str(output_path), index=False)
    print("Archivo guardado correctamente.")
    return df

In [10]:
# Descargar datos históricos. Llamar a la función
raw_all_data = descargar_datos_bigquery_histórico()

Iniciando conexión con BigQuery...
Conexión establecida.
Conexión establecida.


Rango fechas final: 2023-01-02 00:00:00 - 2025-09-14 00:00:00
Guardando archivo en c:\Workspace\mlops_fleca_project\data\raw\raw_data_bq_forecasting_20250914.parquet ...
Archivo guardado correctamente.


In [11]:
# Verificar los datos descargados
raw_all_data_sorted = raw_all_data.sort_values('fecha')
raw_all_data_sorted.tail(5)

,fecha,n_factura,zona_de_venta,producto,familia,cantidad,base_imponible,tipo_IVA,total
373278,2025-09-14,T/207001,LL5,PETIT XOCO (UNID),BOLLERIA,1.0,0.91,10.0,1.0
373277,2025-09-14,T/206979,S4,PETIT XOCO (UNID),BOLLERIA,1.0,0.91,10.0,1.0
373276,2025-09-14,T/206978,S4,PETIT XOCO (UNID),BOLLERIA,1.0,0.91,10.0,1.0
373296,2025-09-14,T/206999,T2,PETIT XOCO (UNID),BOLLERIA,2.0,2.00,10.0,2.2
372313,2025-09-14,T/206959,T5,CAFE LLET,CAFES,1.0,1.73,10.0,1.9


In [12]:
# Cargar el histórico descargado al Feature Group de Hopsworks
from src.data_utils import transformar_a_series_temporales
from src import config
import hopsworks
import pandas as pd

# Usar el DataFrame descargado previamente
raw_all_data = descargar_datos_bigquery_histórico()

# Transformar a series temporales semanales para la familia BOLLERIA
df_ts_historico = transformar_a_series_temporales(raw_all_data, familia="BOLLERIA")

# Ajustar tipos para coincidir con el schema del Feature Group
if 'fecha' in df_ts_historico.columns:
    df_ts_historico = df_ts_historico.drop(columns=['fecha'])
df_ts_historico['year'] = df_ts_historico['year'].astype('int64')
df_ts_historico['week'] = df_ts_historico['week'].astype('int64')
df_ts_historico['familia'] = df_ts_historico['familia'].astype('string')
df_ts_historico['base_imponible'] = df_ts_historico['base_imponible'].astype('float64')
df_ts_historico['is_summer_peak'] = df_ts_historico['is_summer_peak'].astype('int32')
df_ts_historico['is_easter'] = df_ts_historico['is_easter'].astype('int64')
df_ts_historico['week_start'] = pd.to_datetime(df_ts_historico['week_start'])

print(df_ts_historico.dtypes)
print(df_ts_historico.head())

# Conectar a hopsworks y al Feature Store
project = hopsworks.login(api_key_value=config.HOPSWORKS_API_KEY, project=config.HOPSWORKS_PROJECT_NAME)
feature_store = project.get_feature_store()
feature_group = feature_store.get_or_create_feature_group(
    name=config.FEATURE_GROUP_NAME,
    version=config.FEATURE_GROUP_VERSION,
    primary_key=["familia", "week_start"],
    event_time="week_start"
)

# Insertar los datos históricos en el Feature Group
feature_group.insert(
    df_ts_historico,
    write_options={'wait_for_job': True},
)
print("Carga inicial del histórico completada.")

Iniciando conexión con BigQuery...
Conexión establecida.


Rango fechas final: 2023-01-02 00:00:00 - 2025-09-14 00:00:00
Guardando archivo en c:\Workspace\mlops_fleca_project\data\raw\raw_data_bq_forecasting_20250914.parquet ...
Archivo guardado correctamente.
year                       int64
week                       int64
familia           string[python]
base_imponible           float64
is_summer_peak             int32
is_easter                  int64
dias_semana                int64
week_start        datetime64[ns]
dtype: object
   year  week   familia  base_imponible  is_summer_peak  is_easter  \
0  2023     1  BOLLERIA          825.11               0          0   
1  2023     2  BOLLERIA          658.40               0          0   
2  2023     3  BOLLERIA          741.40               0          0   
3  2023     4  BOLLERIA          653.64               0          0   
4  2023     5  BOLLERIA          680.46               0          0   

   dias_semana week_start  
0            7 2023-01-02  
1            7 2023-01-09  
2            7 



To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'


2025-09-21 20:29:24,320 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1242272


Uploading Dataframe: 100.00% |██████████| Rows 137/137 | Elapsed Time: 00:00 | Remaining Time: 00:00


Launching job: times_series_bolleria_feature_group_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1242272/jobs/named/times_series_bolleria_feature_group_1_offline_fg_materialization/executions
2025-09-21 20:30:10,774 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-09-21 20:30:13,999 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-09-21 20:32:06,538 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2025-09-21 20:32:06,700 INFO: Waiting for log aggregation to finish.
2025-09-21 20:32:28,838 INFO: Execution finished successfully.
Carga inicial del histórico completada.


In [13]:
# Comprobar si la semana del 15/09/2025 y sus lags están presentes y son utilizables para predicción
import pandas as pd
from src.data_utils import transformar_features_target

# Calcular la semana objetivo y los lags
semana_objetivo = pd.Timestamp('2025-09-15')
lags = [1, 2, 52]

# Transformar el histórico a features y target
features_and_target = transformar_features_target(
    df_ts_historico,
    lags_list=lags,
    columna_target='base_imponible',
    cols_exogenas=['is_summer_peak', 'is_easter'],
    eliminar_nulos=True,
    return_format='dataframe'
)

print("Semanas disponibles para predicción:")
print(features_and_target[['week_start', 'target']].tail(10))

if semana_objetivo in features_and_target['week_start'].values:
    print(f"La semana objetivo {semana_objetivo.date()} está disponible para predicción.")
else:
    print(f"La semana objetivo {semana_objetivo.date()} NO está disponible para predicción.")
    # Comprobar si faltan semanas para los lags
    ultima_semana = df_ts_historico['week_start'].max()
    faltan_semanas = []
    for lag in lags:
        semana_requerida = semana_objetivo - pd.Timedelta(weeks=lag)
        if semana_requerida not in df_ts_historico['week_start'].values:
            faltan_semanas.append(semana_requerida)
            print(f"FALTA la semana requerida para lag {lag}: {semana_requerida.date()}")
        else:
            print(f"OK: Semana requerida para lag {lag} presente: {semana_requerida.date()}")
    if not faltan_semanas:
        print("Todas las semanas necesarias para los lags están presentes.")
    else:
        print(f"Faltan las siguientes semanas para los lags: {[d.date() for d in faltan_semanas]}")

2025-09-21 20:32:28,845 INFO: Usando DataFrame de entrada: (137, 8)
2025-09-21 20:32:28,848 INFO: Retornando DataFrame combinado: (84, 7)
Semanas disponibles para predicción:
    week_start   target
126 2025-06-30  1308.74
127 2025-07-07  1299.57
128 2025-07-14  1318.98
129 2025-07-21  1408.04
130 2025-07-28  1609.03
131 2025-08-04  1782.56
132 2025-08-11  1741.72
133 2025-08-18  1402.02
134 2025-08-25  1101.07
135 2025-09-01   969.84
La semana objetivo 2025-09-15 NO está disponible para predicción.
OK: Semana requerida para lag 1 presente: 2025-09-08
OK: Semana requerida para lag 2 presente: 2025-09-01
OK: Semana requerida para lag 52 presente: 2024-09-16
Todas las semanas necesarias para los lags están presentes.


In [14]:
# Revisar si están los registros de la semana del 8 al 14 de septiembre de 2025 en el histórico
import pandas as pd

# Definir el rango de fechas de la semana
fecha_inicio = pd.Timestamp('2025-09-08')
fecha_fin = pd.Timestamp('2025-09-14')

# Filtrar registros en el DataFrame original de ventas
registros_semana = raw_all_data[(pd.to_datetime(raw_all_data['fecha']) >= fecha_inicio) & (pd.to_datetime(raw_all_data['fecha']) <= fecha_fin)]
print(f"Registros encontrados en raw_all_data para la semana del 8 al 14/09/2025: {len(registros_semana)}")
print(registros_semana.head())

# Filtrar registros en el DataFrame de series temporales
registros_ts_semana = df_ts_historico[df_ts_historico['week_start'] == fecha_inicio]
print(f"Registros en df_ts_historico para week_start={fecha_inicio.date()}: {len(registros_ts_semana)}")
print(registros_ts_semana)

if len(registros_semana) == 0:
    print("No hay registros de ventas en raw_all_data para esa semana. Revisa la descarga desde BigQuery.")
if len(registros_ts_semana) == 0:
    print("No se generó la serie temporal para esa semana. Revisa la función de agregación semanal.")

Registros encontrados en raw_all_data para la semana del 8 al 14/09/2025: 2483
             fecha     n_factura zona_de_venta            producto  \
370724  2025-09-14  No facturado         SALON  7 CEREALS FORMATGE   
370725  2025-09-14  No facturado         SALON  7 CEREALS FORMATGE   
370726  2025-09-10  No facturado         SALON  7 CEREALS FORMATGE   
370727  2025-09-12  No facturado       TERRAZA      7 CEREALS FUET   
370728  2025-09-11  No facturado         BARRA      7 CEREALS FUET   

           familia  cantidad  base_imponible  tipo_IVA  total  
370724  BOCADILLOS       1.0            2.41      10.0   2.65  
370725  BOCADILLOS       1.0            2.41      10.0   2.65  
370726  BOCADILLOS       1.0            2.41      10.0   2.65  
370727  BOCADILLOS       1.0            2.41      10.0   2.65  
370728  BOCADILLOS       1.0            2.41      10.0   2.65  
Registros en df_ts_historico para week_start=2025-09-08: 1
     year  week   familia  base_imponible  is_summer_peak

In [15]:
import pandas as pd

# Semanas requeridas para los lags de la predicción de la semana 15/09/2025
semanas_lag = [
    pd.to_datetime("2025-09-08"),  # lag 1
    pd.to_datetime("2025-09-01"),  # lag 2
    pd.to_datetime("2025-08-25"),  # lag 3
    pd.to_datetime("2024-09-16"),  # lag 52
]

print("Semanas requeridas para lags:")
for semana in semanas_lag:
    existe = semana in pd.to_datetime(df_ts_historico["week_start"]).values
    print(f"{semana.date()}: {'Presente' if existe else 'Faltante'}")


Semanas requeridas para lags:
2025-09-08: Presente
2025-09-01: Presente
2025-08-25: Presente
2024-09-16: Presente
